In [35]:
import pandas as pd
import joblib
from google.colab import files
from sklearn.preprocessing import StandardScaler
from sklearn.svm import OneClassSVM

In [36]:
uploaded = files.upload()

Saving engine_data.csv to engine_data (2).csv


In [37]:
df = pd.read_csv("engine_data.csv")

In [38]:
features = ["Engine rpm", "Lub oil pressure", "Fuel pressure", "Coolant pressure", "lub oil temp", "Coolant temp"]
X = df[features]

In [39]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [40]:
joblib.dump(scaler, "scaler.pkl")
print("Scaler saved successfully!")

Scaler saved successfully!


In [46]:
#Manual Grid Search for One-Class SVM
nu_values = [0.05, 0.1, 0.15]
gamma_values = ["scale", "auto", 0.01, 0.1, 1.0]

best_model = None
best_score = 0
best_params = {}

for nu in nu_values:
    for gamma in gamma_values:
        oc_svm = OneClassSVM(nu=nu, kernel="rbf", gamma=gamma)
        df["Anomaly_SVM"] = oc_svm.fit_predict(X_scaled)
        df["Anomaly_SVM"] = df["Anomaly_SVM"].map({1: 0, -1: 1})

        # Evaluate model
        correct_anomalies = df[(df["Anomaly_SVM"] == 1) & (df["Engine Condition"] == 1)].shape[0]
        score = correct_anomalies  # More anomalies detected in faulty engines is better

        print(f"nu={nu}, gamma={gamma} → Detected {correct_anomalies} faulty engines")

        # Store best model
        if score > best_score:
            best_score = score
            best_model = oc_svm
            best_params = {"nu": nu, "gamma": gamma}

print("\nBest parameters:", best_params)

nu=0.05, gamma=scale → Detected 591 faulty engines
nu=0.05, gamma=auto → Detected 591 faulty engines
nu=0.05, gamma=0.01 → Detected 573 faulty engines
nu=0.05, gamma=0.1 → Detected 597 faulty engines
nu=0.05, gamma=1.0 → Detected 1096 faulty engines
nu=0.1, gamma=scale → Detected 1189 faulty engines
nu=0.1, gamma=auto → Detected 1189 faulty engines
nu=0.1, gamma=0.01 → Detected 1174 faulty engines
nu=0.1, gamma=0.1 → Detected 1184 faulty engines
nu=0.1, gamma=1.0 → Detected 1377 faulty engines
nu=0.15, gamma=scale → Detected 1780 faulty engines
nu=0.15, gamma=auto → Detected 1780 faulty engines
nu=0.15, gamma=0.01 → Detected 1788 faulty engines
nu=0.15, gamma=0.1 → Detected 1792 faulty engines
nu=0.15, gamma=1.0 → Detected 1875 faulty engines

Best parameters: {'nu': 0.15, 'gamma': 1.0}


In [47]:
joblib.dump(best_model, "one_class_svm.pkl")
print("Best model saved successfully!")

Best model saved successfully!
